In [ ]:
import numpy as np 
import pandas as pd
import csv
# from getpass import getpass
from time import sleep
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common import exceptions

In [ ]:
# call the browser that we can control
driver = Chrome()
driver.get('https://www.twitter.com/login')

In [ ]:
# enter the usename or email address for login
username = driver.find_element_by_xpath('//input[@name="username"]')
username.send_keys('myemail@gmail.com')
username.send_keys(Keys.RETURN)

In [ ]:
# enter the password for login
password = driver.find_element_by_xpath('//input[@name="password"]')
password.send_keys('mypassword')
password.send_keys(Keys.RETURN)

In [ ]:
# click the search bar and enter the keyword, this time we are trying to scrap the tweets from Hilary Clinton
search_input = driver.find_element_by_xpath('//input[@aria-label="搜尋條件"]')
search_input.send_keys('HillaryClinton')
search_input.send_keys(Keys.RETURN)

In [ ]:
# we enter the homepage of Hillary Clinton
driver.find_element_by_xpath('//a[@href="/HillaryClinton"]').click()

In [ ]:
# create a function that can scrap the element from a tweet
def get_tweet_data(card):
    username = card.find_element_by_xpath('./div[1]/div[1]/div[1]/div[2]//span').text
    user_id = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    try:
        post_date = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except exceptions.NoSuchElementException:
        return
    
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[2]/div[1]//span').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]/div[1]//span').text
    reply_count = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_count = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_count = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    tweet = (username, user_id, post_date, comment + responding, \
        reply_count, retweet_count, like_count)
    return tweet

In [ ]:
# create a function that control scraping when the browesr automatically scrolls down
def collect_all_tweets_from_current_view(driver, lookback_limit=18):
    page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    if len(page_cards) <= lookback_limit:
        return page_cards
    else:
        return page_cards[-lookback_limit:]

In [ ]:
# start scraping
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = collect_all_tweets_from_current_view(driver)
    for card in page_cards:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)

    scroll_attempt = 0
    
    while True: 
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(3)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            last_position = curr_position
            break
    

In [ ]:
# check how many tweets we successfully scrapped
len(data)

In [ ]:
# save the dataframe to csv file for further use
with open ('dataKH2_tweets.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Tweet', 'Comments', 'Retweets', 'Likes']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)